# Data Wrangling Project - Tweets

This project was developed to analyse tweets from the @WeRateDogs page from Twitter. It aims to go through the role process of data wrangling: gather, acess and clean data.

The project asseses 8 quality issues in the dataset and 2 tidiness issues, which will be corrected. After proper cleaning, the cleanned dataframe is stored in a csv file named `twitter_archive_master.csv`.

An analysis of the dataset to get 3 insights and 1 visualization is also performed.

In [1]:
# Importing libraries to be used
import pandas as pd
import numpy as np
import matplotlib as plt
import tweepy 
import requests
import re
import json
import datetime as dt
%matplotlib inline

# Gather

First, data will be gathered from 3 different sources: a csv file, a tsv file and through JSON data (Twitter's API).

In [2]:
# Checking dataset names in directory
! ls

image-predictions.tsv
tweet-json.zip
tweet_json.txt
twitter-archive-enhanced.csv
wrangle_act.ipynb


In [39]:
# Reading csv  file in a DataFrame
df_tweets = pd.read_csv('twitter-archive-enhanced.csv')

In [40]:
# Verifying datasets were correctly readed
df_tweets.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
# reading tsv file form url
image_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(image_url)

with open (image_url.split('/')[-1], mode='wb') as file:
    file.write(r.content)
    
df_images = pd.read_csv('image-predictions.tsv', sep='\t')
df_images.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [6]:
# Querying tweets from WeRateDogs account

# Defining keys
consumer_key = 'ONUpD9b5r51DhiUgLkXRXrMhA'
consumer_secret = 'O5n2aCGj4Iayu6duX61DmShfd8a5n3bJY9lCcIUgPGRqNJbfq1'
access_token = '1267179330255425539-i30xULQQ0bjslCFTU9Vx4E0O5UGnnP'
access_secret = 'PReOFJUHjtEx5FwsvSfKu8Hj7a4TXAobAbauitOgIxelR'

In [7]:
# Setup Authentication and Tweepy API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
# logs tweets that fail to pull from API
error_list = []

# Searching tweets through tweet_id content and writing tweets to text file

with open('tweet_json.txt', 'w', encoding='utf8') as f:
    for tweet_id in df_tweets.tweet_id:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("{0}: Successful".format(tweet_id))
            json.dump(tweet, f)
            f.write('\n')
        except Exception as e:
            print("{0}: {1}".format(tweet_id, e))
            error_list.append(tweet_id)

892420643555336193: Successful
892177421306343426: Successful
891815181378084864: Successful
891689557279858688: Successful
891327558926688256: Successful
891087950875897856: Successful
890971913173991426: Successful
890729181411237888: Successful
890609185150312448: Successful
890240255349198849: Successful
890006608113172480: Successful
889880896479866881: Successful
889665388333682689: Successful
889638837579907072: Successful
889531135344209921: Successful
889278841981685760: Successful
888917238123831296: Successful
888804989199671297: Successful
888554962724278272: Successful
888202515573088257: [{'code': 144, 'message': 'No status found with that ID.'}]
888078434458587136: Successful
887705289381826560: Successful
887517139158093824: Successful
887473957103951883: Successful
887343217045368832: Successful
887101392804085760: Successful
886983233522544640: Successful
886736880519319552: Successful
886680336477933568: Successful
886366144734445568: Successful
886267009285017600: S

Rate limit reached. Sleeping for: 206


758740312047005698: Successful
758474966123810816: Successful
758467244762497024: Successful
758405701903519748: Successful
758355060040593408: Successful
758099635764359168: Successful
758041019896193024: Successful
757741869644341248: Successful
757729163776290825: Successful
757725642876129280: Successful
757611664640446465: Successful
757597904299253760: Successful
757596066325864448: Successful
757400162377592832: Successful
757393109802180609: Successful
757354760399941633: Successful
756998049151549440: Successful
756939218950160384: Successful
756651752796094464: Successful
756526248105566208: Successful
756303284449767430: Successful
756288534030475264: Successful
756275833623502848: Successful
755955933503782912: Successful
755206590534418437: Successful
755110668769038337: Successful
754874841593970688: Successful
754856583969079297: Successful
754747087846248448: Successful
754482103782404096: Successful
754449512966619136: Successful
754120377874386944: Successful
75401181

In [8]:
# Printing number of successful and fail tweets
print("Successful: {0} tweets".format(len(df_tweets.tweet_id) - len(error_list)))
print("Failed: {0}".format(len(error_list)))

NameError: name 'error_list' is not defined

Most of the tweets that can't be found probably were deleted.

Now that the tweets were extracted, a new dataframe will be created through a list of dictionaries. The dataframe will be created with 5 columns: 'tweet_id', 'favorites', 'retweets', 'retweeted' and 'full_text'

In [9]:
new_df = []

with open('tweet_json.txt', 'r', encoding='utf8') as f:
    for line in f:
        new_line = json.loads(line)
        new_df.append({'tweet_id': new_line['id'],
                        'favorites': new_line['favorite_count'],
                        'retweets': new_line['retweet_count'],
                        'retweeted': new_line['retweeted'],
                        'full_text': new_line['full_text'][:]})
        
tweets_data = pd.DataFrame(new_df)

In [10]:
tweets_data.head(3)

,tweet_id,favorites,retweets,retweeted,full_text
0,892420643555336193,35891,7609,False,This is Phineas. He's a mystical boy. Only eve...
1,892177421306343426,30964,5637,False,This is Tilly. She's just checking pup on you....
2,891815181378084864,23298,3730,False,This is Archie. He is a rare Norwegian Pouncin...


# Assess

In [11]:
# Checking general info from the dataset

df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

It is possible to verify that there is some missing data from 'expanded_urls' column, which is a quality issue.

In [12]:
# Descriptive statistics from df

df_tweets.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
# Checking name of columns from df_tweets

list(df_tweets)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [62]:
# Checking different unique value from 'name' column

df_tweets.name.value_counts()

None        745
a            55
Charlie      12
Lucy         11
Oliver       11
           ... 
Bobby         1
Petrick       1
Buddah        1
Finnegus      1
Ridley        1
Name: name, Length: 957, dtype: int64

In [15]:
# Checking duplicated tweet_id values 

df_tweets[df_tweets.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [16]:
# Checking min and max values of rating numerators

print(df_tweets['rating_numerator'].min())
print(df_tweets['rating_numerator'].max())

0
1776


In [17]:
# Checking data with '0' in nominator

df_tweets.loc[df_tweets['rating_numerator'] == 0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/835152434...,0,10,None,None,None,None,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459...,0,10,None,None,None,None,None


In [18]:
# Checking data with '1776' in nominator

df_tweets.loc[df_tweets['rating_numerator'] == 1776]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,None,None,None,None


In [19]:
# Checking if denominator has only the value '10'

df_tweets['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [46]:
# Checking retweets content

# Filling NaN values with 0 and transforming column to int type
df_tweets.retweeted_status_id = df_tweets.retweeted_status_id.fillna(0).astype(int)
df_tweets.loc[df_tweets['retweeted_status_id'] != 0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,-2147483648,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,-2147483648,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,-2147483648,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,-2147483648,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,-2147483648,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,746521445350707200,NaN,NaN,2016-06-25 01:52:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Shaggy. He knows exactl...,-2147483648,4.196984e+09,2015-11-21 00:46:50 +0000,https://twitter.com/dog_rates/status/667866724...,10,10,Shaggy,None,None,None,None
1043,743835915802583040,NaN,NaN,2016-06-17 16:01:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Extremely intelligent dog here....,-2147483648,4.196984e+09,2015-11-19 00:32:12 +0000,https://twitter.com/dog_rates/status/667138269...,10,10,None,None,None,None,None
1242,711998809858043904,NaN,NaN,2016-03-21 19:31:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @twitter: @dog_rates Awesome Tweet! 12/10. ...,-2147483648,7.832140e+05,2016-03-21 19:29:52 +0000,https://twitter.com/twitter/status/71199827977...,12,10,None,None,None,None,None
2259,667550904950915073,NaN,NaN,2015-11-20 03:51:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @dogratingrating: Exceptional talent. Origi...,-2147483648,4.296832e+09,2015-11-20 03:43:06 +0000,https://twitter.com/dogratingrating/status/667...,12,10,None,None,None,None,None


Analyzing the dataframe above, it can be seen that retweets have and "RT" on the text content.

In [20]:
# Checking name of columns from df_image
list(df_images)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

### Quality

* Timestamp column has "+0000" after time
* Timestamp column is not in correct type
* Exclude retweets
* Exclude dogs with name "a"
* Exclude dogs with name "an"
* Exclude dogs with name "the"
* Set all rating denominators to "10"
* Exclude rating numerator of "1776"

### Tidiness

* Image prediction and tweets dataframes could be merged in the same df.
* Dogs breed can be unified in one column

# Clean

In [82]:
# Making a copy of the dataframe to clean it (best practice)
tweets_clean = df_tweets.copy()
images_clean = df_images.copy()
api_clean = tweets_data.copy()

In [83]:
tweets_clean.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
944,752682090207055872,NaN,NaN,2016-07-12 01:52:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Nothing better than a doggo and a sunset. 10/1...,0,NaN,NaN,https://twitter.com/dog_rates/status/752682090...,10,10,None,doggo,None,None,None
2348,666055525042405380,NaN,NaN,2015-11-16 00:49:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a Siberian heavily armored polar bear ...,0,NaN,NaN,https://twitter.com/dog_rates/status/666055525...,10,10,a,None,None,None,None
932,754011816964026368,NaN,NaN,2016-07-15 17:56:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charlie. He pouts until he gets to go ...,0,NaN,NaN,https://twitter.com/dog_rates/status/754011816...,12,10,Charlie,None,None,None,None


## I - Quality Issues

### Define

Exclude "+0000" from timestamp column by string slicing.

### Code

In [84]:
tweets_clean['timestamp'] = tweets_clean.timestamp.str[:-5]

### Test

In [85]:
tweets_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,0,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,0,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,0,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


### Define

Transform timestamp column in correct type through `to_datetime` function.

### Code

In [86]:
tweets_clean['timestamp'] = pd.to_datetime(tweets_clean['timestamp'])

### Test

In [87]:
tweets_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   int64         
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   timestamp                   2356 non-null   datetime64[ns]
 4   source                      2356 non-null   object        
 5   text                        2356 non-null   object        
 6   retweeted_status_id         2356 non-null   int32         
 7   retweeted_status_user_id    181 non-null    float64       
 8   retweeted_status_timestamp  181 non-null    object        
 9   expanded_urls               2297 non-null   object        
 10  rating_numerator            2356 non-null   int64         
 11  rating_denominator          2356 non-null   int64       

### Define

Exclude retweets from dataframe through `drop` function. As it was observed before, retweets usually have an 'RT' on its text. 

### Code

In [88]:
tweets_clean.drop((tweets_clean[tweets_clean['retweeted_status_id'] != 0].index), inplace=True)

### Test

In [89]:
tweets_clean.loc[tweets_clean['retweeted_status_id'] != 0]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Define

Exclude dogs with name "a" through `drop` function.

### Code

In [90]:
tweets_clean.drop((tweets_clean[tweets_clean['name'] == 'a'].index), inplace=True)

### Test

In [91]:
#tweets_clean.loc[tweets_clean['name'] == 'a']
tweets_clean['name'].value_counts()

None       680
Charlie     11
Lucy        11
Oliver      10
Cooper      10
          ... 
Barclay      1
Mosby        1
Cecil        1
Craig        1
Ridley       1
Name: name, Length: 955, dtype: int64

### Define

Exclude dogs with name "an" through `drop` function.

### Code

In [92]:
tweets_clean.drop((tweets_clean[tweets_clean['name'] == 'an'].index), inplace=True)

### Test

In [93]:
#tweets_clean.loc[tweets_clean['name'] == 'an']
tweets_clean['name'].value_counts()

None       680
Lucy        11
Charlie     11
Cooper      10
Oliver      10
          ... 
Barclay      1
Mosby        1
Cecil        1
Craig        1
Rumble       1
Name: name, Length: 954, dtype: int64

### Define

Exclude dogs with name "the" through `drop` function.

### Code

In [94]:
tweets_clean.drop((tweets_clean[tweets_clean['name'] == 'the'].index), inplace=True)

### Test

In [95]:
tweets_clean.loc[tweets_clean['name'] == 'the']
#tweets_clean['name'].value_counts()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Define

Setting all denominators as a 10 value, by replacing all values by 10.

### Code

In [96]:
tweets_clean['rating_denominator'] = 10

### Test

In [97]:
tweets_clean.rating_denominator.value_counts()

10    2106
Name: rating_denominator, dtype: int64

### Define

Exclude outlier 1776 of rating_denominator column through `drop` function.

### Code

In [98]:
tweets_clean.drop((tweets_clean[tweets_clean['rating_numerator'] == 1776].index), inplace=True)

### Test

In [99]:
tweets_clean[tweets_clean['rating_numerator'] == 1776]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


After this last code line, it is possible to observe that all quality issues were corrected in the dataframe. 

## II - Tidiness Issues

### Define

 The columns 'doggo', 'floofer', 'pupper' and 'puppo' are variables and should form just 1 column. A new Breed column will be created using the `melt` function.

### Code

In [102]:

tweets_clean = pd.melt(tweets_clean, id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name'], var_name='breed', value_vars=['doggo', 'floofer', 'pupper', 'puppo'])


KeyError: "The following 'value_vars' are not present in the DataFrame: ['doggo', 'floofer', 'pupper', 'puppo']"

In [105]:
del tweets_clean['value']

### Test

In [107]:
tweets_clean.sample(4)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,breed
1882,670755717859713024,NaN,NaN,2015-11-29 00:06:39,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Gin &amp; Tonic. They're having a...,0,NaN,NaN,https://twitter.com/dog_rates/status/670755717...,9,10,Gin,doggo
4144,667491009379606528,NaN,NaN,2015-11-19 23:53:52,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Two dogs in this one. Both are rare Jujitsu Py...,0,NaN,NaN,https://twitter.com/dog_rates/status/667491009...,7,10,None,floofer
6158,669354382627049472,NaN,NaN,2015-11-25 03:18:15,"<a href=""http://twitter.com/download/iphone"" r...",Meet Dug. Dug fucken loves peaches. 8/10 https...,0,NaN,NaN,https://twitter.com/dog_rates/status/669354382...,8,10,Dug,pupper
5474,695051054296211456,NaN,NaN,2016-02-04 01:07:39,"<a href=""http://twitter.com/download/iphone"" r...","Meet Brian (pronounced ""Kirk""). He's not amuse...",0,NaN,NaN,https://twitter.com/dog_rates/status/695051054...,6,10,Brian,pupper


### Define

Image predictions and tweet dataframes can be merged in the same dataframe. This will be donw through `merge` function.

### Code

In [108]:
tweets_clean = pd.merge(tweets_clean, images_clean, on=['tweet_id'], how='left')

### Test

In [109]:
tweets_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'breed', 'jpg_url', 'img_num', 'p1',
       'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf',
       'p3_dog'],
      dtype='object')

# Analysis and Visualization

In [110]:
# Storing values
tweets_clean.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

In [111]:
df_clean = pd.read_csv('twitter_archive_master.csv', encoding='utf-8')

In [112]:
df_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,0,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,0,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,0,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True


# References

* [Fill dataframe with for loop](https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop/28058264#28058264)
* [Drop rows with certain condition](https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/)
* [Melt Function to create breed column](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html)